## Objective

Now that I know that my submodels perform a bit better than the model on the entire dataset, I can take advantage of the submodels and find the coefficients that are most associated with response variable.

In [1]:
CATEGORY_GROUPS_IN_QUESTION = [['Pick up Dead Animal', 'Animal Generic Request'],
['Abandoned Vehicles', 'Abandoned Bicycle'],
['Rodent Activity',	'Bed Bugs', 'Mice Infestation - Residential'],
['Sidewalk Repair', 'Sidewalk Repair (Make Safe)'],
['Needle Pickup'],
['Unsatisfactory Living Conditions', 'Poor Conditions of Property', 'Unsanitary Conditions - Establishment', 'Illegal Occupancy', 'Heat - Excessive  Insufficient'],
['Request for Pothole Repair'],
['Graffiti Removal']]

## Objective

Does this have statistically significant coefs, assuming homoskedacity and a linear predictor-response relationship and Normalized residuals and imperfect collinearity?

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
from tqdm import tqdm

from utilities import remove_one_feature

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20, 5

import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))
from helper_functions import dummify_cols_and_baselines, make_alphas, remove_outliers_by_type, adjusted_r2, transform_school, get_vifs

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

In [5]:
df_orig = transform_school(df_orig)
df_orig.shape

../preprocessing/helper_functions.py:58: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  df.school = df.school.str.extract(r'(\d\d?)').astype(int)


(516406, 40)

## Filterering by `TYPE`

In [6]:
i = 7
print CATEGORY_GROUPS_IN_QUESTION[i]
df_orig = df_orig[df_orig.TYPE.isin(CATEGORY_GROUPS_IN_QUESTION[i])]
df_orig.shape

['Graffiti Removal']


(16674, 40)

## Removing outliers

A standard procedure is to remove values further than 3 standard deviations from the mean. Since I have so many low values and some very high values, I anecdotally think that the low values are very likely to be true, and the high values not so much.

So, I will remove values further than 3 SDs from the median, by type.

Ideally, I would take into account the time dimension. I would like to do so given more time.

In [7]:
df_outliers_removed = remove_outliers_by_type(df_orig, y_col='COMPLETION_HOURS_LOG_10')
df_outliers_removed.shape

(16199, 40)

## Remove `TYPE` col

In [8]:
df_outliers_removed.drop('TYPE', axis=1, inplace=True)

## Choosing columns

In [9]:
['race_black',
 'race_asian',
 'race_other',
 'earned_income_per_capita',
 'poverty_pop_w_public_assistance',
 'poverty_pop_w_food_stamps',
 'poverty_pop_w_ssi',
 'school',
 'school_std_dev',
 'bedroom_std_dev',
 'value',
 'value_std_dev',
 'rent',
 'income',
 'Source_Constituent Call']

['SubmittedPhoto',
 'race_asian',
 'race_hispanic',
 'race_other',
 'poverty_pop_below_poverty_level',
 'earned_income_per_capita',
 'poverty_pop_w_food_stamps',
 'school',
 'bedroom',
 'rent',
 'income_std_dev',
 'Source_Constituent Call']

In [79]:
cols_orig_dataset = ['COMPLETION_HOURS_LOG_10', 'SubmittedPhoto', 'Property_Type', 'Source']
cols_census = [
     'race_black',
     'race_asian',
     'race_hispanic',
     'race_other',
     'housing',
     'bedroom',
     'rent_std_dev',
]
cols_engineered = ['queue_wk', 'queue_wk_open', 'is_description']

In [80]:
df = df_outliers_removed[cols_orig_dataset + cols_census + cols_engineered]
df.shape

(16199, 14)

## Removing NAs for cols like `school_std_dev`

In [81]:
aa = df.isnull().any().reset_index()
nas = aa[aa[0] == True]['index']
print nas

10    rent_std_dev
Name: index, dtype: object


In [82]:
# this is a bad temporary band-aid
df = df.dropna(subset=nas.tolist())
df.shape

(15890, 14)

## Dummify

In [83]:
cols_to_dummify = [i for i in df.dtypes[df.dtypes == object].index if i != 'TYPE']
cols_to_dummify

['Property_Type', 'Source', 'housing']

In [84]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify, chosen_col_i=2)

Intersection is baseline 0 3
Self Service is baseline 1 3
own is baseline 2 3


In [85]:
df_dummified.shape

(15890, 14)

## Checking for multicollinearity

In [96]:
df_dummified.head(1).T

,905400
COMPLETION_HOURS_LOG_10,0.0124857
SubmittedPhoto,False
poverty_pop_below_poverty_level,0.262473
bedroom,2
queue_wk,12873
queue_wk_open,1
is_description,True


In [18]:
get_vifs(df_dummified.drop(['SubmittedPhoto', 'is_description'], axis=1), 'COMPLETION_HOURS_LOG_10')

poverty_pop_w_food_stamps          4.236079
earned_income_per_capita           3.316667
poverty_pop_below_poverty_level    3.245716
housing_std_dev                    2.737470
race_hispanic                      2.603753
income                             2.565994
poverty_pop_w_ssi                  2.333123
race_black                         2.215186
school                             2.213382
race_asian                         2.083619
school_std_dev                     2.021305
Source_Citizens Connect App        2.017575
Source_Constituent Call            1.941944
income_std_dev                     1.780807
value_std_dev                      1.756591
rent_std_dev                       1.675506
rent                               1.662569
poverty_pop_w_public_assistance    1.647938
bedroom                            1.421333
bedroom_std_dev                    1.419842
value                              1.417218
queue_wk                           1.156319
race_other                      

## Running model

In [19]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import string
from StringIO import StringIO


In [20]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop('COMPLETION_HOURS_LOG_10', axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

## Use LassoCV to find col subsets

In [22]:
pipe = make_pipeline(StandardScaler(), LassoCV())
cv = ShuffleSplit(X_train.shape[0], n_iter=1, test_size=0.2, random_state=300)

In [26]:
params = {'lassocv__alphas': make_alphas(-2, -1)}
model = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=cv, verbose=0)
model.fit(X_train, y_train)
pd.DataFrame(model.cv_results_).T.iloc[2:5]

,0,1,2
mean_test_score,0.105206,0.0947433,0.0384517
mean_train_score,0.100853,0.0889941,0.0342303
param_lassocv__alphas,[0.01],[0.03],[0.1]


In [27]:
model.best_params_

{'lassocv__alphas': [0.01]}

In [28]:
'{} cols go to zero out of {}'.format(
    len(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0]),
    len(X_train.columns)
)

'15 cols go to zero out of 27'

In [29]:
cols_zero = list(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0])
cols_zero

['race_black',
 'race_asian',
 'race_other',
 'earned_income_per_capita',
 'poverty_pop_w_public_assistance',
 'poverty_pop_w_food_stamps',
 'poverty_pop_w_ssi',
 'school',
 'school_std_dev',
 'bedroom_std_dev',
 'value',
 'value_std_dev',
 'rent',
 'income',
 'Source_Constituent Call']

## Use subsetted cols to run lin reg

In [86]:
df_dummified.columns = [col.translate(None, string.punctuation).replace(' ', '') if col != 'COMPLETION_HOURS_LOG_10' else col for col in df_dummified.columns]

In [87]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop(['COMPLETION_HOURS_LOG_10'], axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

In [88]:
col_list = ' + '.join(df_dummified.drop(['COMPLETION_HOURS_LOG_10'], axis=1).columns)

est = smf.ols(
    'COMPLETION_HOURS_LOG_10 ~ {}'.format(col_list), 
    pd.concat([X_train, y_train], axis=1)).fit()

In [40]:
est.summary().tables[0]

Dep. Variable:,COMPLETION_HOURS_LOG_10,R-squared:,0.104
Model:,OLS,Adj. R-squared:,0.102
Method:,Least Squares,F-statistic:,91.66
Date:,"Thu, 23 Feb 2017",Prob (F-statistic):,4.43e-286
Time:,18:37:22,Log-Likelihood:,-9842.2
No. Observations:,12712,AIC:,1.972e+04
Df Residuals:,12695,BIC:,1.985e+04
Df Model:,16,,
Covariance Type:,nonrobust,,


### Getting adjusted $R^2$ on test set

In [89]:
y_pred = est.predict(X_test)

In [90]:
adjusted_r2(y_test, y_pred, num_features=X_test.shape[1])

0.10524100967255721

In [91]:
mean_squared_error(y_test, y_pred)**0.5

0.51836019581545267

## Interpreting model

Which features are most associated with completion time?

In [97]:
df_results = pd.read_csv(StringIO(est.summary().tables[1].as_csv()), index_col=0).reset_index()
df_results.columns = ['coef_name'] + [i.rstrip().lstrip() for i in df_results.columns][1:]
df_results.coef_name = df_results.coef_name.map(lambda x: x.strip())
df_results = df_results.sort_values('P>|t|')
df_results['pct_diff_for_y'] = (10**df_results.coef - 1) * 100
df_results['pct_diff_for_y_abs'] = pd.np.abs((10**df_results.coef - 1) * 100)
df_results.sort_values('pct_diff_for_y_abs', inplace=True, ascending=False)
df_results.shape

(14, 8)

In [96]:
df_results

,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],pct_diff_for_y,pct_diff_for_y_abs
0,Intercept,1.975500,0.034000,58.554,0.000,1.909 2.042,9351.483924,9351.483924
8,rentstddev,0.592500,0.218000,2.721,0.007,0.166 1.019,291.291127,291.291127
6,raceother,-0.161100,0.120000,-1.341,0.180,-0.397 0.074,-30.991911,30.991911
4,raceasian,0.106900,0.034000,3.167,0.002,0.041 0.173,27.908675,27.908675
5,racehispanic,0.103800,0.029000,3.523,0.000,0.046 0.162,26.998912,26.998912
1,SubmittedPhoto[T.True],0.090300,0.017000,5.457,0.000,0.058 0.123,23.111890,23.111890
12,SourceCitizensConnectApp,-0.106900,0.018000,-6.076,0.000,-0.141 -0.072,-21.819220,21.819220
11,PropertyTypeAddress,0.075800,0.011000,7.013,0.000,0.055 0.097,19.069355,19.069355
2,isdescription[T.True],0.066600,0.010000,6.501,0.000,0.047 0.087,16.573544,16.573544
13,SourceConstituentCall,-0.024400,0.021000,-1.162,0.245,-0.066 0.017,-5.463395,5.463395


In [94]:
scores = []

for col in X_train.columns:
    if col != 'Intercept':
        score = remove_one_feature([col], df_dummified)
        scores.append((col, score))
        
sorted(scores, key=lambda x: x[1])[::-1]        

[('raceblack', 34.559),
 ('SourceCitizensConnectApp', 34.31),
 ('SourceConstituentCall', 34.185),
 ('raceother', 34.091),
 ('bedroom', 32.199),
 ('PropertyTypeAddress', 32.178),
 ('SubmittedPhoto', 32.053),
 ('queuewkopen', 31.988),
 ('racehispanic', 31.873),
 ('queuewk', 31.796),
 ('rentstddev', 31.769),
 ('raceasian', 31.741),
 ('isdescription', 30.017)]

In [98]:
df_results[df_results['P>|t|'] < 0.1][df_results.coef_name != 'Intercept']

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],pct_diff_for_y,pct_diff_for_y_abs
8,rentstddev,0.592500,0.218000,2.721,0.007,0.166 1.019,291.291127,291.291127
4,raceasian,0.106900,0.034000,3.167,0.002,0.041 0.173,27.908675,27.908675
5,racehispanic,0.103800,0.029000,3.523,0.000,0.046 0.162,26.998912,26.998912
1,SubmittedPhoto[T.True],0.090300,0.017000,5.457,0.000,0.058 0.123,23.111890,23.111890
12,SourceCitizensConnectApp,-0.106900,0.018000,-6.076,0.000,-0.141 -0.072,-21.819220,21.819220
11,PropertyTypeAddress,0.075800,0.011000,7.013,0.000,0.055 0.097,19.069355,19.069355
2,isdescription[T.True],0.066600,0.010000,6.501,0.000,0.047 0.087,16.573544,16.573544
7,bedroom,0.014900,0.007000,2.258,0.024,0.002 0.028,3.490384,3.490384
10,queuewkopen,0.000700,0.000032,20.492,0.000,0.001 0.001,0.161311,0.161311
9,queuewk,0.000009,0.000001,8.105,0.000,6.67e-06 1.09e-05,0.002027,0.002027


# Selected Findings

- **Increased diversity in rent** associated with **worse** completion time.
- **Asian and Hispanic** associated with **worse** completion time.
- **From app** associated with **better** completion time.

### Weird Findings
- **Submitted photo** associated with **worse** completion time.
- **More bedrooms** associated with **worse** completion time.